In [ ]:
WITH
	states_counties_index AS (SELECT * FROM states_counties)
SELECT
	SC_IDX.state,
	DATE_PART_YEAR(releasedate) AS year,
	AVG(GREATEST(NONE, D0, D1, D2, D3, D4)) AS max_drought_coeff
FROM
	droughts,
    states_counties_index AS SC_IDX
WHERE
	SC_IDX.index = droughts.STATE_COUNTY
GROUP BY
	SC_IDX.state,
	year


In [ ]:
WITH
	drought_coeffs AS (
      	SELECT
      		GREATEST(NONE, D0, D1, D2, D3, D4) AS max_drought_coeff
        FROM
        	droughts
    )
SELECT
	state_county,
    DATE_PART_YEAR(releasedate) AS year,
	MAX(max_drought_coeff)
FROM
	droughts,
    drought_coeffs
GROUP BY
	state_county,
	year

In [ ]:
WITH states_counties_index AS 
(
    SELECT
        * 
    FROM
        states_counties 
)
,
states_abbrv AS 
(
    SELECT
        * 
    FROM
        states_abbrv 
)
,
yearly_aqi AS
(
    SELECT
        SC_IDX.state,
        DATE_PART_YEAR(AQI.date_of_last_change) AS year,
        AVG(AQI.aqi) AS AVG_AQ 
    FROM
        airquality AS AQI,
        states_counties_index AS SC_IDX 
    WHERE
        SC_IDX.index = AQI.STATE_COUNTY 
    GROUP BY
        SC_IDX.state,
        year 
)
,
yearly_temperatures AS 
(
    SELECT
        states_abbrv.abbr AS state_abbreviated,
        DATE_PART_YEAR(dt) AS year,
        AVG(averagetemperature) AS avg_temp 
    FROM
        temperatures,
        states_abbrv 
    WHERE
        states_abbrv.state = temperatures.state 
    GROUP BY
        state_abbreviated,
        year 
    ORDER BY
        state_abbreviated,
        year 
)
,
yearly_wildfires AS 
(
    SELECT
        SC_IDX.state AS state,
        FIRE_YEAR,
        COUNT(*) AS total 
    FROM
        wildfires AS WF,
        states_counties_index AS SC_IDX 
    WHERE
        SC_IDX.index = WF.STATE_COUNTY 
    GROUP BY
        SC_IDX.state,
        FIRE_YEAR 
    ORDER BY
        SC_IDX.state ASC,
        FIRE_YEAR ASC 
)
,
yearly_droughts AS 
(
    SELECT
        SC_IDX.state,
        DATE_PART_YEAR(releasedate) AS year,
        AVG(GREATEST(NONE, D0, D1, D2, D3, D4)) AS avg_drought_coeff 
    FROM
        droughts,
        states_counties_index AS SC_IDX 
    WHERE
        SC_IDX.index = droughts.STATE_COUNTY 
    GROUP BY
        SC_IDX.state,
        year 
    ORDER BY
        SC_IDX.state ASC,
        year ASC 
)
SELECT
    yearly_aqi.state,
    yearly_aqi.year,
    yearly_aqi.avg_aq,
    yearly_temperatures.avg_temp,
    yearly_wildfires.total,
    yearly_droughts.avg_drought_coeff 
FROM
    yearly_aqi 
    JOIN
        yearly_temperatures 
        ON yearly_aqi.state = yearly_temperatures.state_abbreviated 
        AND yearly_aqi.year = yearly_temperatures.year 
    JOIN
        yearly_wildfires 
        ON yearly_aqi.state = yearly_wildfires.state 
        AND yearly_aqi.year = yearly_wildfires.FIRE_YEAR 
    JOIN
        yearly_droughts 
        ON yearly_aqi.state = yearly_droughts.state 
        AND yearly_aqi.year = yearly_droughts.year 
ORDER BY
    yearly_aqi.state ASC,
    yearly_aqi.year ASC